<a href="https://colab.research.google.com/github/jansiddiqui/Learning-MachineLearning/blob/main/Prct_4_Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WorkFlow
- News Data --> Data pre processing --> Train Test Split --> Logistic Regression Model


1 --> Fake News

0 --> Real News

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/WELFake_Dataset.csv', engine='python', on_bad_lines='skip')
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11941 entries, 0 to 11940
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  11941 non-null  object
 1   title       11858 non-null  object
 2   text        11915 non-null  object
 3   label       11921 non-null  object
dtypes: object(4)
memory usage: 373.3+ KB


In [ ]:
df.shape

(11941, 4)

In [ ]:
df.describe()

,Unnamed: 0,title,text,label
count,11941,11858,11915,11921
unique,11941,11600,11520,4
top,11920,ATTENTION BAD GUYS With Weapons: There Are NO ...,,1
freq,1,3,126,6271


In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
title,83
text,26
label,20


In [ ]:
# replacing the null values with empty string
df = df.fillna('')

In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [ ]:
#merging the title and text
df['content'] = df['title']+' '+df['text']

In [ ]:
print(df['content'])

0        LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1           Did they post their votes for Hillary already?
2        UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3        Bobby Jindal, raised Hindu, uses story of Chri...
4        SATAN 2: Russia unvelis an image of its terrif...
                               ...                        
11936    CONSERVATIVE LEADER Sends Stern Message To Mit...
11937    MALE PAKISTANI IMMIGRANT Dresses As Woman In B...
11938    America’s looming freak show: How GOP control ...
11939    Cosmology/ Love & the Afterlife   Cosmology/ L...
11940    Mattis says North Korea missile test meant to ...
Name: content, Length: 11941, dtype: object


In [ ]:
#Separating the data & label
X = df.drop(columns='label', axis=1)
Y = df['label']

In [ ]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0              0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1              1                                                      
2              2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3              3  Bobby Jindal, raised Hindu, uses story of Chri...   
4              4  SATAN 2: Russia unvelis an image of its terrif...   
...          ...                                                ...   
11936      11916  CONSERVATIVE LEADER Sends Stern Message To Mit...   
11937      11917  MALE PAKISTANI IMMIGRANT Dresses As Woman In B...   
11938      11918  America’s looming freak show: How GOP control ...   
11939      11919                    Cosmology/ Love & the Afterlife   
11940      11920  Mattis says North Korea missile test meant to ...   

                                                    text  \
0      No comment is expected from Barack Obama Membe...   
1         Did they post the

Stemming:- It is the process of reducing a word to its root word.

Ex- actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [30]:
df['content'] = df['content'].apply(stemming)

In [31]:
print(df['content'])

0        law enforc high alert follow threat cop white ...
1                                post vote hillari alreadi
2        unbeliev obama attorney gener say charlott rio...
3        bobbi jindal rai hindu use stori christian con...
4        satan russia unv imag terrifi new supernuk wes...
                               ...                        
11936    conserv leader send stern messag mitt romney g...
11937    male pakistani immigr dress woman burqa lure y...
11938    america loom freak show gop control terror nat...
11939    cosmolog love afterlif cosmolog love afterlif ...
11940    matti say north korea missil test meant provok...
Name: content, Length: 11941, dtype: object


In [32]:
#seaparating the data and label
X = df['content'].values
Y = df['label'].values

In [33]:
print(X)

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video comment expect barack obama member fyf fukyoflag blacklivesmatt movement call lynch hang white peopl cop encourag radio show tuesday night turn tide kill white peopl cop send messag kill black peopl america one f yoflag organ call sunshin radio blog show host texa call sunshin f ing opinion radio show snapshot fyf lolatwhitefear twitter page p show urg support call fyf tonight continu dismantl illu white snapshot twitter radio call invit fyf radio show air p eastern standard time show caller clearli call lynch kill white peopl minut clip radio show heard provid breitbart texa someon would like refer hannib alreadi receiv death threat result interrupt fyf confer call unidentifi black man said mother f ker start f ing like us bunch ni er takin one us roll said cau alreadi roll gang anyway six seven black mother f cker see white person lynch ass let turn tabl conspir cop start lose peopl state emerg specul 

In [34]:
print(Y)

['1' '1' '1' ... '0' '1' '0']


In [35]:
Y.shape


(11941,)

Splitting the dataset to training & test data

In [49]:
#converting the texual data to numerical data
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)


In [50]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2258070 stored elements and shape (11941, 64923)>
  Coords	Values
  (0, 31818)	0.005918790072209571
  (0, 17065)	0.01174965160681638
  (0, 24683)	0.0083054398389805
  (0, 1373)	0.023645896255438042
  (0, 19739)	0.005573838950182306
  (0, 56873)	0.02005783372384497
  (0, 11594)	0.11028569781396481
  (0, 62436)	0.05373067883027244
  (0, 6092)	0.0874800398732227
  (0, 20642)	0.6005783880061522
  (0, 56393)	0.024558326750974597
  (0, 60905)	0.011590066607973952
  (0, 10975)	0.0068345207378806795
  (0, 18105)	0.007430630678284518
  (0, 4454)	0.010008448598714203
  (0, 40210)	0.005674062182378421
  (0, 35601)	0.005948157043312581
  (0, 20487)	0.08579691257230745
  (0, 37395)	0.03804538643161722
  (0, 8208)	0.035056864236555074
  (0, 33621)	0.07841059529333587
  (0, 23660)	0.06906416619798349
  (0, 42694)	0.028013156788110744
  (0, 16997)	0.014563673077322412
  (0, 46054)	0.09240496838579437
  :	:
  (11940, 38631)	0.041285014530714

In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

Training the model : Logistic Regression

In [53]:
model = LogisticRegression()

In [54]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [55]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [57]:
print("Accuracy score of the training data: ",training_data_accuracy)

Accuracy score of the training data:  0.9742462311557789


In [58]:
# Accuracy score on the training data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [59]:
print("Accuracy score of the test data: ", test_data_accuracy)

Accuracy score of the test data:  0.911678526580159


Making a Predictive System

In [64]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

# Convert prediction[0] to an integer for correct comparison
if (int(prediction[0]) == 0):
  print('The news is Real')
else:
  print('The news is Fake')

['0']
The news is Real


In [65]:
print(Y_test[0])

0
